<a href="https://colab.research.google.com/github/Mingyang0816/Startup-Founders-Graph-Analysis/blob/main/Mentors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mentors Notebook

*   Mentors are defined to be the researcher in the **last** position of each journal (usually PhD mentors, or an influential figure in the research field).
*   Collate the list of mentors for all startup founders, and web-scrape their information from their Google Scholar profile pages.
*   Build **mentor database** using these information, used for further analysis and identification of potential founders to invest in.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
# Load data
founder_pubs = pd.read_excel("founder_pubs.xlsx")

In [ ]:
# Convert year into strings
founder_pubs["year"] = founder_pubs["year"].astype(str)

founder_pubs.head(5)

,researcher,title,authors,position,publication,year,num_citations,citation_link
0,Leonard Charles Jarrott,Burning plasma achieved in inertial fusion,"['AB Zylstra', 'OA Hurricane', 'DA Callahan', ...",7,Nature 601,2022,400,https://scholar.google.com/scholar?oi=bibs&hl=...
1,Leonard Charles Jarrott,Constraints on sub-GeV dark-matter–electron sc...,"['P Agnes', 'IFM Albuquerque', 'T Alexander', ...",7,Physical review letters 121,2018,274,https://scholar.google.com/scholar?oi=bibs&hl=...
2,Leonard Charles Jarrott,Lawson criterion for ignition exceeded in an i...,"['H Abu-Shawareb', 'R Acree', 'P Adams', 'J Ad...",7,Physical review letters 129,2022,271,https://scholar.google.com/scholar?oi=bibs&hl=...
3,Leonard Charles Jarrott,Focusing of short-pulse high-intensity laser-a...,"['T Bartal', 'ME Foord', 'C Bellei', 'MH Key',...",7,Nature Physics 8,2012,170,https://scholar.google.com/scholar?oi=bibs&hl=...
4,Leonard Charles Jarrott,Design of inertial fusion implosions reaching ...,"['AL Kritcher', 'CV Young', 'HF Robey', 'CR We...",7,Nature Physics 18,2022,136,https://scholar.google.com/scholar?oi=bibs&hl=...


In [ ]:
# Convert authors column into lists
founder_pubs["authors"] = founder_pubs["authors"].apply(lambda x: ast.literal_eval(x))

founder_pubs.head(5)

,researcher,title,authors,position,publication,year,num_citations,citation_link
0,Leonard Charles Jarrott,Burning plasma achieved in inertial fusion,"[AB Zylstra, OA Hurricane, DA Callahan, AL Kri...",7,Nature 601,2022,400,https://scholar.google.com/scholar?oi=bibs&hl=...
1,Leonard Charles Jarrott,Constraints on sub-GeV dark-matter–electron sc...,"[P Agnes, IFM Albuquerque, T Alexander, AK Alt...",7,Physical review letters 121,2018,274,https://scholar.google.com/scholar?oi=bibs&hl=...
2,Leonard Charles Jarrott,Lawson criterion for ignition exceeded in an i...,"[H Abu-Shawareb, R Acree, P Adams, J Adams, B ...",7,Physical review letters 129,2022,271,https://scholar.google.com/scholar?oi=bibs&hl=...
3,Leonard Charles Jarrott,Focusing of short-pulse high-intensity laser-a...,"[T Bartal, ME Foord, C Bellei, MH Key, KA Flip...",7,Nature Physics 8,2012,170,https://scholar.google.com/scholar?oi=bibs&hl=...
4,Leonard Charles Jarrott,Design of inertial fusion implosions reaching ...,"[AL Kritcher, CV Young, HF Robey, CR Weber, AB...",7,Nature Physics 18,2022,136,https://scholar.google.com/scholar?oi=bibs&hl=...


In [ ]:
def get_mentor_info(mentor_url, mentee):
    '''
    Retrieve basic information of mentor from Google Scholar profile page.

    Parameters
    ----------
    mentor_url: Google Scholar profile page of specified mentor.
    mentee: Full name of mentee/researcher

    Return
    ------
    mentor_info: dictionary consisting of mentor's name, position, email, and published content.

    '''

    # Parse webpage
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
    }
    response = requests.get(mentor_url, headers = headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Name
    name = soup.select_one("#gsc_prf_in").text

    # Affiliation
    affiliation = soup.select_one("#gsc_prf_inw+ .gsc_prf_il").text

    # Field
    field_ele = soup.select('.gsc_prf_inta')
    if field_ele:
        fields = []
        for element in field_ele:
            fields.append(element.text)
    else:
        fields = "No field"

    # Num citations, h-index and i10-index
    stats_ele = soup.select('.gsc_rsb_std')
    citations = stats_ele[0].text
    h_idx = stats_ele[2].text
    i10_idx = stats_ele[4].text

    # Store each author as dictionary
    mentor_info = {
        "mentee": mentee,
        "name": name,
        "affiliation": affiliation,
        "field": fields,
        "num_citations": int(citations),
        "h_index": int(h_idx),
        "i10_index": int(i10_idx)
    }

    return mentor_info

In [ ]:
def create_mentor_table(mentor_list, mentee):

    # Scrape mentor profiles
    mentor_profiles = []
    for link in mentor_list.values():
        mentor_profiles.append(get_mentor_info(link, mentee))

    # Convert list into dataframe
    mentor_df = pd.DataFrame(mentor_profiles)
    return mentor_df

In [ ]:
# Last authors for Jarrott's journals
jarrott_mentors = {
    "Debra Callahan": "https://scholar.google.com/citations?hl=en&user=UjNsx8UAAAAJ",
    "Farhat Beg": "https://scholar.google.com/citations?user=1T9P0BwAAAAJ&hl=en&oi=ao",
    "Omar Hurricane": "https://scholar.google.com/citations?user=PshpzVUAAAAJ&hl=en&oi=ao",
    "M.S. Wei": "https://scholar.google.com/citations?user=Mlc9XGwAAAAJ&hl=en&oi=ao",
    "Joao Jorge Santos": "https://scholar.google.com/citations?user=PEmNplYAAAAJ&hl=en&oi=sra",
    "Andrew MacPhee": "https://scholar.google.com/citations?user=Fci9tIEAAAAJ&hl=en&oi=ao",
    "Pravesh K Patel": "https://scholar.google.com/citations?hl=en&user=t0WB_IcAAAAJ",
    "Brian Spears": "https://scholar.google.com/citations?user=2JJOVekAAAAJ&hl=en&oi=ao",
    "Tilo Doeppner": "https://scholar.google.com/citations?user=Q8ZDWpcAAAAJ&hl=en&oi=ao",
    "Hong Sio": "https://scholar.google.com/citations?user=DMwrt7QAAAAJ&hl=en&oi=ao",
    "Pierre A. Michel": "https://scholar.google.com/citations?user=yQ0rg1kAAAAJ&hl=en&oi=ao",
    "Christopher R Weber": "https://scholar.google.com/citations?user=R8uA1zRUGCoC&hl=en&oi=ao",
    "Marilyn B. Schneider": "https://scholar.google.com/citations?user=mQI2yacAAAAJ&hl=en&oi=ao",
    "Justin Wark": "https://scholar.google.com/citations?hl=en&user=6zmwGSYAAAAJ",
    "Steven Rose": "https://scholar.google.com/citations?user=8eSFZDkAAAAJ&hl=en&oi=ao",
    "Hideaki Habara": "https://scholar.google.com/citations?user=Fy31B_cAAAAJ&hl=en&oi=ao",
    "S. Austin Yi": "https://scholar.google.com/citations?hl=en&user=awBh2VEAAAAJ",
    "Gizzi Leonida Antonio": "https://scholar.google.com/citations?hl=en&user=QghWW2sAAAAJ",
    "Otto Landen": "https://scholar.google.com/citations?user=t392b7gAAAAJ&hl=en&oi=ao",
    "Richard Stephens": "https://scholar.google.com/citations?user=lAP4rWMAAAAJ&hl=en&oi=sra",
    "Kazuo A. Tanaka": "https://scholar.google.com/citations?user=QoauxRsAAAAJ&hl=en&oi=ao"
}

In [ ]:
# Mentor table for Jarrott
jarrott_mentors_df = create_mentor_table(jarrott_mentors, "Leonard Charles Jarrott")
jarrott_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Leonard Charles Jarrott,Debra Callahan,Focused Energy,"[inertial fusion, high-energy density, lasers]",17401,71,194
1,Leonard Charles Jarrott,Farhat Beg,Unknown affiliation,No field,17181,62,216
2,Leonard Charles Jarrott,Omar Hurricane,Inertial Confinement Fusion Program Chief Scie...,"[Plasma Physics, High Energy Density Physics, ...",11259,59,137
3,Leonard Charles Jarrott,M.S. Wei,University of Rochester Laboratory for Laser E...,"[laser plasma interaction, high-energy-density...",7382,43,98
4,Leonard Charles Jarrott,Joao Jorge Santos,"Univ. Bordeaux - CNRS - CEA, CELIA UMR 5107, F...","[laser-plasma physics, inertial fusion, high e...",5368,39,90
5,Leonard Charles Jarrott,Andrew MacPhee,Lawrence Livermore National Laboratory,"[Plasma physics, ICF, filling in forms]",14301,62,179
6,Leonard Charles Jarrott,Pravesh K Patel,Lawrence Livermore National Laboratory,[physics],21851,76,230
7,Leonard Charles Jarrott,Brian Spears,"Physicist, Lawrence Livermore National Laboratory","[fusion, inertial confinement, machine learnin...",9912,56,115
8,Leonard Charles Jarrott,Tilo Doeppner,Staff scientist at the NIF/ Lawrence Livermore...,"[High energy density physics, Inertial Confine...",17442,71,224
9,Leonard Charles Jarrott,Hong Sio,Lawrence Livermore National Laboratory,"[ICF, HED physics, nuclear diagnostics]",2675,30,66


In [ ]:
# Last authors for Draycott's journals
draycott_mentors = {
    "Alice Ting": "https://scholar.google.com/citations?user=MiT99qEAAAAJ&hl=en&oi=ao",
    "Wendy V Gilbert": "https://scholar.google.com/citations?user=NRYE068AAAAJ&hl=en&oi=ao",
    "Luisa Escobar-Hoyos": "https://scholar.google.com/citations?user=iXjIQRIAAAAJ&hl=en&oi=ao"
}

In [ ]:
# Mentor table for Draycott
draycott_mentors_df = create_mentor_table(draycott_mentors, "Austin Draycott")
draycott_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Austin Draycott,Alice Ting,Stanford,No field,31460,71,118
1,Austin Draycott,Wendy V Gilbert,Department of Molecular Biophysics & Biochemis...,"[RNA, gene expression, translation, epitranscr...",5604,33,37
2,Austin Draycott,Luisa Escobar-Hoyos,"Assistant Professor, Yale University","[Cancer therapy and biomarkers, RNA biology, I...",1285,15,21


In [ ]:
# Last authors for Weinberg's journals
weinberg_mentors = {
    "Jeff Coller": "https://scholar.google.com/citations?user=wSJc28sAAAAJ&hl=en&oi=ao",
    "David Bartel": "https://scholar.google.com/citations?user=aLegzvoAAAAJ&hl=en&oi=ao",
    "Dinshaw J. Partel": "https://scholar.google.com/citations?user=OSxYKeQAAAAJ&hl=en&oi=ao",
    "Eckhard Jankowsky": "https://scholar.google.com/citations?user=GIXpuksAAAAJ&hl=en&oi=ao",
    "Imran S Haque": "https://scholar.google.com/citations?user=Cp_wDj4AAAAJ&hl=en&oi=ao",
    "Jonathan S. Weissman": "https://scholar.google.com/citations?user=YY7Rql4AAAAJ&hl=en&oi=ao",
    "Sandra L. Wolin": "https://scholar.google.com/citations?user=C0QZx_8AAAAJ&hl=en&oi=ao",
    "John David Gross": "https://scholar.google.com/citations?user=hKKUhugAAAAJ&hl=en&oi=ao",
    "Danica Galonic Fujimori": "https://scholar.google.com/citations?user=B133j-sAAAAJ&hl=en&oi=ao"
}

In [ ]:
# Mentor table for Weinberg
weinberg_mentors_df = create_mentor_table(weinberg_mentors, "David Weinberg")
weinberg_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,David Weinberg,Jeff Coller,Johns Hopkins University,[RNA],9530,35,45
1,David Weinberg,David Bartel,"Professor of Biology, HHMI, MIT, Whitehead Ins...","[Biology, Gene Regulation, MicroRNAs, Noncodin...",246489,131,219
2,David Weinberg,Dinshaw J. Patel,Memorial Sloan-Kettering Cancer Center,[Structural Biology],80602,150,548
3,David Weinberg,Eckhard Jankowsky,MODERNA,No field,12502,50,85
4,David Weinberg,Imran S Haque,Recursion Pharmaceuticals,"[Genetics, Computational Biology, Artificial I...",6928,29,39
5,David Weinberg,Jonathan S. Weissman,UCSF/HHMI,No field,134696,157,294
6,David Weinberg,Sandra L. Wolin,National Cancer Institute,"[RNA biology, noncoding RNAs, RNA surveillance]",11486,56,84
7,David Weinberg,John David Gross,"Professor of Pharmaceutical Chemistry, UCSF","[Structural Biology, mRNA translation, mRNA de...",11843,42,69
8,David Weinberg,Danica Galonic Fujimori,UCSF,"[Biochemistry, Chemical Biology, Ribosome, Chr...",9933,33,53


In [ ]:
# Last authors for Carlson's journals
carlson_mentors = {
    "David O. Morgan": "https://scholar.google.com/citations?user=cVhK91EAAAAJ&hl=en&oi=ao",
    "Huilin Zhou": "https://scholar.google.com/citations?user=2UlzVoAAAAAJ&hl=en&oi=sra",
    "Jennifer Doudna": "https://scholar.google.com/citations?user=YO5XSXwAAAAJ&hl=en&oi=ao"
}

In [ ]:
# Mentor table for Carlson
carlson_mentors_df = create_mentor_table(carlson_mentors, "Christopher R Carlson")
carlson_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Christopher R Carlson,David O. Morgan,"University of California, San Francisco","[Cell cycle, mitosis, APC, Cdk]",44240,87,140
1,Christopher R Carlson,Huilin Zhou,UCSD,[genome maintenance],10164,41,59
2,Christopher R Carlson,Jennifer Doudna,Professor of Molecular and Cell Biology and Ch...,"[CRISPR-Cas, RNA biology, gene editing]",139164,156,432


In [ ]:
# Last authors for Kocherga's journals
kocherga_mentors = {
    "Thomas A. Schmedake": "https://scholar.google.com/citations?user=lpaelkYAAAAJ&hl=en&oi=ao",
    "Michael G. Walter": "https://scholar.google.com/citations?user=tPDMWC4AAAAJ&hl=en&oi=ao",
    "Yong Zhang": "https://scholar.google.com/citations?user=p7-nPv8AAAAJ&hl=en&oi=sra",
    "Christopher Bejger": "https://scholar.google.com/citations?user=4jw6-tUAAAAJ&hl=en&oi=ao",
    "Tino Hofmann": "https://scholar.google.com/citations?user=--PVdvAAAAAJ&hl=en&oi=sra"
}

In [ ]:
# Mentor table for Kocherga
kocherga_mentors_df = create_mentor_table(kocherga_mentors, "Margaret Kocherga")
kocherga_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Margaret Kocherga,Thomas A. Schmedake,Unknown affiliation,No field,2469,17,25
1,Margaret Kocherga,Michael G. Walter,"Professor of Chemistry, University of North Ca...","[Solar Energy Conversion, Porphyrins, Photoche...",13096,22,29
2,Margaret Kocherga,Yong Zhang,"UNC-Charlotte, NREL, Dartmouth, Xiamen Univ.","[physics, material, semiconductor, optics]",10754,48,157
3,Margaret Kocherga,Christopher Bejger,"Associate Professor of Chemistry, UNC Charlotte",No field,458,10,10
4,Margaret Kocherga,Tino Hofmann,University of North Carolina at Charlotte,"[THz-IR-VUV Ellipsometry, Optical Hall Effect,...",4004,36,87


In [ ]:
# Last authors for Wong's journals
wong_mentors = {
    "Ethan Garner": "https://scholar.google.com/citations?user=6YdmM2wAAAAJ&hl=en&oi=ao",
    "James J Collins": "https://scholar.google.com/citations?user=te5DjPQAAAAJ&hl=en&oi=sra",
    "Lars David Renner": "https://scholar.google.com/citations?user=kllIhfwAAAAJ&hl=en&oi=ao",
    "Cesar de la Fuente": "https://scholar.google.com/citations?user=N2OdcFYAAAAJ&hl=en&oi=sra"
}

In [ ]:
# Mentor table for Wong
wong_mentors_df = create_mentor_table(wong_mentors, "Felix Wong")
wong_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Felix Wong,Ethan Garner,Harvard,"[cytoskeleton, prokaryotic cell biology, micro...",12835,38,52
1,Felix Wong,James J Collins,"MIT; Wyss Institute, Harvard; Broad Institute ...","[synthetic biology, antibiotics, biological ph...",134023,172,365
2,Felix Wong,Lars David Renner,"Group Leader, Leibniz Institute of Polymer Res...","[Microbiology, Chemical Biology, Lipid Bilayer...",3153,26,34
3,Felix Wong,César de la Fuente,University of Pennsylvania,"[Synthetic Biology, Artificial Intelligence, M...",13001,54,117


In [ ]:
# Last authors for Wilson's journals
wilson_mentors = {
    "Ned S. Wingreen": "https://scholar.google.com/citations?user=jDdbJHcAAAAJ&hl=en&oi=sra",
    "Zemer Gitai": "https://scholar.google.com/citations?user=Ff8o0AUAAAAJ&hl=en&oi=ao",
    "Jared Toettcher": "https://scholar.google.com/citations?user=GOOFRX0AAAAJ&hl=en&oi=sra",
    "Mohammad R. Seyedsayamdost": "https://scholar.google.com/citations?user=nSgAHXkAAAAJ&hl=en&oi=sra",
    "Kenneth S. Kosik": "https://scholar.google.com/citations?user=4jg-_CUAAAAJ&hl=en&oi=ao",
    "Dzwokai Ma": "https://scholar.google.com/citations?user=eGec18gAAAAJ&hl=en&oi=sra",
    "Diego Acosta-Alvear": "https://scholar.google.com/citations?user=jIvxCBoAAAAJ&hl=en&oi=sra",
    "Carolina Arias": "https://scholar.google.com/citations?user=8WnCtSEAAAAJ&hl=en&oi=ao",
    "Mark S. Sherwin": "https://scholar.google.com/citations?user=oKSmNTQAAAAJ&hl=en&oi=ao",
    "Javier Read de Alaniz": "https://scholar.google.com/citations?user=Gumh67cAAAAJ&hl=en&oi=sra",
    "Beth L Pruitt": "https://scholar.google.com/citations?user=C7-K83oAAAAJ&hl=en&oi=ao",
    "Meghan Morrissey": "https://scholar.google.com/citations?user=Kj0N9f8AAAAJ&hl=en&oi=sra",
    "Cesar Carrasco-Lopez": "https://scholar.google.com/citations?user=1dl54CgAAAAJ&hl=en&oi=ao",
    "James Martin": "https://scholar.google.com/citations?user=rUUtArEAAAAJ&hl=en&oi=sra"
}

In [ ]:
# Mentor table for Wilson
wilson_mentors_df = create_mentor_table(wilson_mentors, "Maxwell Z Wilson")
wilson_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Maxwell Z Wilson,Ned S. Wingreen,"Professor of Molecular Biology, Princeton Univ...",No field,38997,90,217
1,Maxwell Z Wilson,Zemer Gitai,"Professor of Molecular Biology, Princeton Univ...",[Microbiology],10731,50,83
2,Maxwell Z Wilson,Jared Toettcher,"Associate Professor, Princeton University","[Cell signaling, developmental patterning, sys...",6698,35,49
3,Maxwell Z Wilson,Mohammad R. Seyedsayamdost,Princeton University,"[Biochemistry, Microbiology, Chemistry]",7363,53,105
4,Maxwell Z Wilson,Kenneth S. Kosik,UC Santa Barbara,[kosik@lifesci.ucsb.edu],56490,122,300
5,Maxwell Z Wilson,Dzwokai Ma,UCSB,"[cell biology, virology]",5960,25,27
6,Maxwell Z Wilson,Diego Acosta-Alvear,"Principal Investigator, Altos Labs, Bay Area I...","[Stress biology, cellular health]",5120,25,29
7,Maxwell Z Wilson,Carolina Arias,Chan Zuckerberg BioHub,"[Virology, cell biology, drug development]",3115,16,17
8,Maxwell Z Wilson,Mark S. Sherwin,"Professor of Physics, University of California...","[condensed matter physics, terahertz science a...",9654,46,110
9,Maxwell Z Wilson,Javier Read de Alaniz,University of California Santa Barbara,No field,10109,50,111


In [ ]:
# Last authors for Foresti's journals
foresti_mentors = {
    "Dimos Poulikakos": "https://scholar.google.com/citations?user=RQZI4r0AAAAJ&hl=en&oi=ao",
    "Jennifer A. Lewis": "https://scholar.google.com/citations?user=Wdny3bkAAAAJ&hl=en&oi=sra",
    "Aldo Ferrari": "https://scholar.google.com/citations?user=W2uegOMAAAAJ&hl=en&oi=sra"
}

In [ ]:
# Mentor table for Foresti
foresti_mentors_df = create_mentor_table(foresti_mentors, "Daniele Foresti")
foresti_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Daniele Foresti,Dimos Poulikakos,ETH Zurich,"[Energy, Nanotechnology, Thermodynamics, Inter...",41880,107,430
1,Daniele Foresti,Jennifer A. Lewis,Harvard University,"[3D printing, soft materials, printed electron...",64175,109,264
2,Daniele Foresti,Aldo Ferrari,Chief Scientific Officer - Hylomorph AG,[Biomedicine],5921,38,65


In [ ]:
# Last authors for Diller's journals
diller_mentors = {
    "Steve Collins": "https://scholar.google.com/citations?user=eF5vfBAAAAAJ&hl=en&oi=sra",
    "Carmel Majidi": "https://scholar.google.com/citations?user=1LyndUsAAAAJ&hl=en&oi=ao"
}

In [ ]:
# Mentor table for Diller
diller_mentors_df = create_mentor_table(diller_mentors, "Stuart Diller")
diller_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Stuart Diller,Steve Collins,Stanford University,"[exoskeletons, prosthetics, biomechanics, robo...",14478,47,82
1,Stuart Diller,Carmel Majidi,"Professor of Mechanical Engineering, Carnegie ...","[Soft Robotics, Soft Machines, Stretchable Ele...",23839,79,202


In [ ]:
# Last authors for Haynes's journals
haynes_mentors = {
    "Daniel Koditschek": "https://scholar.google.com/citations?user=K_PZFhEAAAAJ&hl=en&oi=sra",
    "Carl Wellington": "https://scholar.google.com/citations?user=Pqd_-nkAAAAJ&hl=en&oi=ao",
    "Alfred A. Rizzi": "https://scholar.google.com/citations?user=GAwdhxEAAAAJ&hl=en&oi=sra",
    "Alexander Styler": "https://scholar.google.com/citations?user=lqr4UR8AAAAJ&hl=en&oi=sra",
    "Anthony Stentz": "https://scholar.google.com/citations?user=cnFhxvMAAAAJ&hl=en&oi=sra",
    "Andrew J. Spence": "https://scholar.google.com/citations?user=WU5KHp8AAAAJ&hl=en&oi=sra",
    "Siddhartha Srinivasa": "https://scholar.google.com/citations?user=RCi98EAAAAAJ&hl=en&oi=sra",
    "Nemanja Djuric": "https://scholar.google.com/citations?user=ykWqS0YAAAAJ&hl=en&oi=sra",
    "Tzu-Kuo Huang": "https://scholar.google.com/citations?user=vDsqNQgAAAAJ&hl=en&oi=ao",
    "Charles Robert Hogg III": "https://scholar.google.com/citations?user=FgVA4dMAAAAJ&hl=en&oi=ao",
    "Neil Traft": "https://scholar.google.com/citations?user=gmNv_Z8AAAAJ&hl=en&oi=sra"
}

In [ ]:
# Mentor table for Haynes
haynes_mentors_df = create_mentor_table(haynes_mentors, "Galen Clark Haynes")
haynes_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Galen Clark Haynes,Daniel Koditschek,Professor of Electrical and Systems Engineerin...,"[Robotics, Dynamical Systems]",25755,71,219
1,Galen Clark Haynes,Carl Wellington,"Head of Autonomy, Aurora Innovation","[Computer Vision, Machine Learning, Self-Drivi...",2312,22,29
2,Galen Clark Haynes,Alfred A. Rizzi,CTO at Boston Dynamics AI Institute,[Robotics],7980,45,92
3,Galen Clark Haynes,Alexander Styler,Carnegie Mellon University,"[Prediction, Optimization, Learning, Energy, V...",287,8,8
4,Galen Clark Haynes,Anthony Stentz,"Courtesy Faculty, Robotics Institute, Carnegie...","[Robotics, Artificial Intelligence, Self-drivi...",29266,82,217
5,Galen Clark Haynes,Andrew J. Spence,Professor,"[Biomechanics, Animal Locomotion, Neuromechani...",3532,27,39
6,Galen Clark Haynes,Siddhartha Srinivasa,"Professor, University of Washington","[Robotics, Motion Planning, Robotic Manipulati...",28388,82,264
7,Galen Clark Haynes,Nemanja Djuric,"Principal Tech Lead Manager, Aurora Innovation","[Artificial Intelligence, Machine Learning, Da...",5765,31,49
8,Galen Clark Haynes,Tzu-Kuo Huang,"Research Scientist, Google DeepMind",[Machine Learning],2627,14,19
9,Galen Clark Haynes,Charles Robert Hogg III,"Software Engineer, Aurora Innovation","[C++, TDD, Gaussian Processes, Geometric Algeb...",360,7,6


In [ ]:
# Last authors for Fontana's journals
fontana_mentors = {
    "Sheref M. Mansy": "https://scholar.google.com/citations?user=T5_VNycAAAAJ&hl=en&oi=ao",
    "James Carothers": "https://scholar.google.com/citations?user=sZD2OecAAAAJ&hl=en&oi=sra",
    "Vincent Noireaux": "https://scholar.google.com/citations?user=lY7nGvsAAAAJ&hl=en&oi=sra"
}

In [ ]:
# Mentor table for Fontana
fontana_mentors_df = create_mentor_table(fontana_mentors, "Jason Fontana")
fontana_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Jason Fontana,Sheref S. Mansy,"Professor of Biochemistry, University of Trento","[prebiotic chemistry, protocells, origins of l...",5222,36,61
1,Jason Fontana,James Carothers,"University of Washington, Seattle, WA",[RNA Synthetic Biology and Metabolic Engineering],3890,21,31
2,Jason Fontana,Vincent Noireaux,University of Minnesota,"[synthetic biology, biological physics, soft m...",13634,48,79


In [ ]:
# Last authors for Barkhudarov's journals
barkhudarov_mentors = {"Martial Hebert": "https://scholar.google.com/citations?user=0ytii2EAAAAJ&hl=en&oi=sra"}

In [ ]:
# Mentor table for Barkhudarov
barkhudarov_mentors_df = create_mentor_table(barkhudarov_mentors, "Narek Melik-Barkhudarov")
barkhudarov_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Narek Melik-Barkhudarov,martial hebert,CMU,"[Computer Vision, Robotics]",64954,122,387


In [ ]:
# Last authors for Lewis's journals
lewis_mentors = {"Auinash Kalsotra": "https://scholar.google.com/citations?user=Jjo08aAAAAAJ&hl=en&oi=sra"}

In [ ]:
# Mentor table for Lewis
lewis_mentors_df = create_mentor_table(lewis_mentors, "Cole J. T. Lewis")
lewis_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Cole J. T. Lewis,Auinash Kalsotra,"Professor of Biochemistry, University of Illin...","[Post-transcriptional gene regulation, Tissue ...",4719,32,46


In [ ]:
# Last authors for Hsing's journals
hsing_mentors = {"David Perreault": "https://scholar.google.com/citations?user=DGqqwrQAAAAJ&hl=en&oi=sra"}

In [ ]:
# Mentor table for Hsing
hsing_mentors_df = create_mentor_table(hsing_mentors, "Mitchell Hsing")
hsing_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Mitchell Hsing,David Perreault,"Professor of Electrical Engineering, MIT","[Power Electronics, Energy, Electronics]",18081,71,229


In [ ]:
# Last authors for Yager's journals
yager_mentors = {"Gultekin Gulsen": "https://scholar.google.com/citations?user=-Q8yFHsAAAAJ&hl=en&oi=sra"}

In [ ]:
# Mentor table for Yager
yager_mentors_df = create_mentor_table(yager_mentors, "David Sparkman-Yager")
yager_mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,David Sparkman-Yager,Gultekin Gulsen,"Associate Professor , University of California...",[Optical Molecular Imaging],2474,26,55


In [ ]:
# Merge mentor tables for all researchers
mentors_df = pd.concat([jarrott_mentors_df, draycott_mentors_df, weinberg_mentors_df, carlson_mentors_df, kocherga_mentors_df,
                        wong_mentors_df, wilson_mentors_df, foresti_mentors_df, diller_mentors_df, haynes_mentors_df,
                        fontana_mentors_df, barkhudarov_mentors_df, lewis_mentors_df, hsing_mentors_df, yager_mentors_df])
mentors_df = mentors_df.reset_index(drop = True)
mentors_df.head(30)

,mentee,name,affiliation,field,num_citations,h_index,i10_index
0,Leonard Charles Jarrott,Debra Callahan,Focused Energy,"[inertial fusion, high-energy density, lasers]",17401,71,194
1,Leonard Charles Jarrott,Farhat Beg,Unknown affiliation,No field,17181,62,216
2,Leonard Charles Jarrott,Omar Hurricane,Inertial Confinement Fusion Program Chief Scie...,"[Plasma Physics, High Energy Density Physics, ...",11259,59,137
3,Leonard Charles Jarrott,M.S. Wei,University of Rochester Laboratory for Laser E...,"[laser plasma interaction, high-energy-density...",7382,43,98
4,Leonard Charles Jarrott,Joao Jorge Santos,"Univ. Bordeaux - CNRS - CEA, CELIA UMR 5107, F...","[laser-plasma physics, inertial fusion, high e...",5368,39,90
5,Leonard Charles Jarrott,Andrew MacPhee,Lawrence Livermore National Laboratory,"[Plasma physics, ICF, filling in forms]",14301,62,179
6,Leonard Charles Jarrott,Pravesh K Patel,Lawrence Livermore National Laboratory,[physics],21851,76,230
7,Leonard Charles Jarrott,Brian Spears,"Physicist, Lawrence Livermore National Laboratory","[fusion, inertial confinement, machine learnin...",9912,56,115
8,Leonard Charles Jarrott,Tilo Doeppner,Staff scientist at the NIF/ Lawrence Livermore...,"[High energy density physics, Inertial Confine...",17442,71,224
9,Leonard Charles Jarrott,Hong Sio,Lawrence Livermore National Laboratory,"[ICF, HED physics, nuclear diagnostics]",2675,30,66


In [ ]:
# Save dataframe as Excel file
mentors_df.to_excel("mentors.xlsx", index = False)